In [ ]:
import os

from dotenv import load_dotenv

from eodhd_ez import EODHDEZClient

load_dotenv()

api_key = os.environ.get('EODHD_API_KEY')
ez = EODHDEZClient(api_key)

exchanges = ez.get_exchanges_df(filters={
	'CountryISO3': 'USA'
})
exchanges

In [ ]:
import json
import os
from typing import Any

import inflection

repo_dir = os.path.dirname(os.getcwd())
assert os.path.exists(repo_dir)

notebooks_dir = os.path.join(repo_dir, 'notebooks')
assert os.path.exists(notebooks_dir)

file = os.path.join(notebooks_dir, 'fundamental_data.json')

if os.path.exists(file):
	with open(file, 'r') as f:
		data = json.load(f)

keys = data.keys()

financials = data['Financials']

reports = financials['Income_Statement']
quarterly = reports['quarterly']
yearly = reports['yearly']

# compare the keys of the quarterly and yearly balance sheet reports
quarterly_keys = set([v for k, v in quarterly.items()][0].keys())
yearly_keys = set([v for k, v in yearly.items()][0].keys())
print(f'delta: {list(quarterly_keys - yearly_keys)}\n')

quarterly_reports = [v for k, v in quarterly.items()]

kv_types = {}


def cast_type(value: Any):
	if value is None:
		return ['NoneType']
	try:
		_num = float(v)
		return ['float']
	except ValueError:
		try:
			_num = int(v)
			return ['int']
		except ValueError:
			return ['str']


for report in quarterly_reports:
	for k, v in report.items():
		if k not in kv_types:
			kv_types[k] = cast_type(v)
		else:
			kv_types[k] = list(set(kv_types[k] + cast_type(v)))

for k, v in kv_types.items():
	if 'NoneType' in v:
		v.remove('NoneType')

for k, v in kv_types.items():
	if len(v) == 0:
		kv_types[k] = ['str']

print('\n'.join([
	f'{inflection.underscore(k)}: Optional[{v[0]}]'
	for k, v in kv_types.items()
]))

# empty_keys = [k for k, v in kv_types.items() if len(v) == 0]

# unique_types = set([t for v in kv_types.values() for t in v])

# pprint(unique_types)